# Courses Demo
This Jupyter notebook is for exploring the data set courses20-21.json
which consists of all Brandeis courses in the 20-21 academic year (Fall20, Spr21, Sum21) 
which had at least 1 student enrolled.

First we need to read the json file into a list of Python dictionaries

In [60]:
import json
from collections import defaultdict

In [61]:
with open("courses20-21.json","r",encoding='utf-8') as jsonfile:
    courses = json.load(jsonfile)

## Structure of a course
Next we look at the fields of each course dictionary and their values

In [62]:
print('there are',len(courses),'courses in the dataset')
print('here is the data for course 1246')
courses[1246]

there are 7813 courses in the dataset
here is the data for course 1246


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ['An', 'Huang', 'anhuang@brandeis.edu'],
 'coinstructors': [],
 'code': ['MATH', '223A'],
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

## Cleaning the data
If we want to sort courses by instructor or by code, we need to replace the lists with tuples (which are immutable lists)

In [63]:
for course in courses:
        course['instructor'] = tuple(course['instructor'])
        course['coinstructors'] = tuple([tuple(f) for f in course['coinstructors']])
        course['code']= tuple(course['code'])

In [64]:
print('notice that the instructor and code are tuples now')
courses[1246]

notice that the instructor and code are tuples now


{'limit': 28,
 'times': [{'start': 1080, 'end': 1170, 'days': ['w', 'm']}],
 'enrolled': 4,
 'details': 'Instruction for this course will be offered remotely. Meeting times for this course are listed in the schedule of classes (in ET).',
 'type': 'section',
 'status_text': 'Open',
 'section': '1',
 'waiting': 0,
 'instructor': ('An', 'Huang', 'anhuang@brandeis.edu'),
 'coinstructors': (),
 'code': ('MATH', '223A'),
 'subject': 'MATH',
 'coursenum': '223A',
 'name': 'Lie Algebras: Representation Theory',
 'independent_study': False,
 'term': '1203',
 'description': "Theorems of Engel and Lie. Semisimple Lie algebras, Cartan's criterion. Universal enveloping algebras, PBW theorem, Serre's construction. Representation theory. Other topics as time permits. Usually offered every second year.\nAn Huang"}

# Exploring the data set
Now we will show how to use straight python to explore the data set and answer some interesting questions. Next week we will start learning Pandas/Numpy which are packages that make it easier to explore large dataset efficiently.

Here are some questions we can try to asnwer:
* what are all of the subjects of courses (e.g. COSI, MATH, JAPN, PHIL, ...)
* which terms are represented?
* how many instructors taught at Brandeis last year?
* what were the five largest course sections?
* what were the five largest courses (where we combine sections)?
* which are the five largest subjects measured by number of courses offered?
* which are the five largest courses measured by number of students taught?
* which course had the most sections taught in 20-21?
* who are the top five faculty in terms of number of students taught?
* etc.

In [65]:
#Part 5a
#how many faculty taught COSI courses last year?
newlist = len({course['instructor'] for course in courses if course['subject'] == "COSI"})
print(newlist)

#Part 5b 
#what is the total number of students taking COSI courses last year?
enrolled_in_cs = len({course['enrolled'] for course in courses if course['subject'] == "COSI"})
print(enrolled_in_cs)

#Part 5c 
#what was the median size of a COSI course last year (counting only those courses with at least 10 students)
avgr_cs = sorted({course['enrolled'] for course in courses if course['subject'] == "COSI" if course['enrolled'] >9 })
med = len(avgr_cs)//2
print(avgr_cs[med])

#Part 5d 
#create a list of tuples (E,S) where S is a subject and E is the number of students enrolled in courses in that subject, sort it and print the top 10. This shows the top 10 subjects in terms of number of students taught.
tup_list = sorted({ (course['enrolled'], course['subject']) for course in courses},reverse = True)
print(tup_list[:10])

#Part 5e 
# do the same as in (d) but print the top 10 subjects in terms of number of courses offered
from collections import defaultdict
num_of_classes = defaultdict(int)
for course in courses: num_of_classes[course['subject']] += 1
new_dict= sorted(num_of_classes, reverse = True)

print(new_dict[:10])

# Part 5f
#do the same as (d) but print the top 10 subjects in terms of number of faculty teaching courses in that subject
from collections import defaultdict
num_of_classes_per_instruct = defaultdict(int)
for course in courses: num_of_classes_per_instruct[course['instructor']] += 1
d= sorted(num_of_classes, reverse = True)

print(d[:10]) 

#Part 5.G
num_students_per_prof = defaultdict(int)
for course in courses: num_students_per_prof[course['instructor']] += course['enrolled']
ordered_instructors = [item[0] for item in num_students_per_prof.items()]
ordered_instructors.sort(key = lambda instructor: num_students_per_prof[instructor], reverse = True)
top20instructors = ordered_instructors[:20]
# for instructor in top20instructors: print(instructor, num_students_per_prof[instructor])

#Part 5.E
num_students_per_course = defaultdict(int)
for course in courses: num_students_per_course[course['code']] += course['enrolled']
ordered_course = [item[0] for item in num_students_per_course.items()]
ordered_course.sort(key = lambda course: num_students_per_course[course], reverse = True)
top20course = ordered_course[:20]
# for course in top20course: print(course, num_students_per_course[course])

#Zach's Part 5.F
#Find the Least Popular CS classes
cs_course = [item[0] for item in num_students_per_course.items() if item[0][0] == 'COSI']
cs_course.sort(key = lambda course: num_students_per_course[course])
bottom20course = cs_course[:20]
#for course in bottom20course: print(course, num_students_per_course[course])

#Ava Part 5.F
#Find the Least Popular Biology classes
cs_course = [item[0] for item in num_students_per_course.items() if item[0][0] == 'BIOL']
cs_course.sort(key = lambda course: num_students_per_course[course])
bottom20course = cs_course[:20]
#for course in bottom20course: print(course, num_students_per_course[course])


#for course in bottom20course: print(course, num_students_per_course[course])



('MATH', '10A') 280
('MATH', '15A') 204
('MATH', '10B') 125
('MATH', '20A') 121
('MATH', '23B') 95
('MATH', '36A') 87
('MATH', '8A') 85
('MATH', '37A') 80
('MATH', '35A') 65
('MATH', '36B') 59
('MATH', '121A') 52
('MATH', '40A') 49
('MATH', '16B') 46
('MATH', '124A') 43
('MATH', '126A') 40
('MATH', '5A') 36
('MATH', '28B') 25
('MATH', '401D') 23
('MATH', '3A') 19
('MATH', '115A') 18
